In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
from src.utils.stats_utils import *


In [3]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [4]:
features_class = UnitDataProcessing(config)

IMPORTING: CBE2022 with version CBE2022_16. 
CBE2022 with version CBE2022_16 loaded. 
Paradata shape: (4667834, 27) Questionnaires shape: (122, 36) Microdata shape: (2833981, 41) 
Data Loaded
Paradata Processed
Items Build
Unit Build


In [5]:
self = features_class

In [6]:
df_item = features_class.df_item

Processing f__answer_changed...
Processing f__answer_position...
Processing f__answer_selected...
Processing f__first_decimal...
Processing f__gps...
Processing f__numeric_response...


In [7]:
# PROCES DATA IN A PIVOT TABLE
feature_name = ['f__gps_latitude', 'f__gps_longitude', 'f__gps_accuracy']
data, index_col = self.get_clean_pivot_table(feature_name, remove_low_freq_col=False)

def replace_with_feature_name(columns, feature_names):
    for i, s in enumerate(columns):
        for sub in feature_names:
            if sub in s:
                columns[i] = sub
                break
    return columns

data.columns = replace_with_feature_name(list(data.columns), feature_name)
data = data.reset_index()

In [8]:

# Everything that has 0,0 as coordinates is an outlier
data['s__gps_spatial_extreme_outlier'] = data['f__gps_latitude'].apply(lambda x: 1 if x == 0.000000 else 0)
data['s__gps_spatial_extreme_outlier'] = data['f__gps_longitude'].apply(lambda x: 1 if x == 0.000000 else 0)




mask = (data['s__gps_spatial_extreme_outlier'] < 1)
# Convert to cartesian for faster computation
data.loc[mask,'x'], data.loc[mask,'y'], data.loc[mask,'z'] = lat_lon_to_cartesian(data[mask]['f__gps_latitude'],
                                                       data[mask]['f__gps_longitude'])
median_x = data[mask].drop_duplicates(subset='x')['x'].median()
median_y = data[mask].drop_duplicates(subset='y')['y'].median()
median_z = data[mask].drop_duplicates(subset='z')['z'].median()

# Calculate distances from the median point
data['distance_to_median'] = np.sqrt((data[mask]['x'] - median_x)**2 + 
                                   (data[mask]['y'] - median_y)**2 +
                                   (data[mask]['z'] - median_z)**2 
                                     )

# Set a threshold (e.g., 95th percentile of distances)
threshold = data[mask]['distance_to_median'].quantile(0.95) + 30

# Everything that is above 30 + the median distance is an outlier
data.loc[mask, 's__gps_spatial_extreme_outlier'] = data[mask]['distance_to_median'] > threshold


# Convert accuracy from meters to kilometers
data['accuracy'] = data['f__gps_accuracy'] / 1e6

# Create KDTree
tree = cKDTree(data[['x', 'y', 'z']])

# Convert 10 meters to kilometers, the unit of the Earth's radius
radius = 10 / 1e6

# Query for counts accounting for accuracy
counts = [len(tree.query_ball_point(xyz, r=radius + accuracy)) - 1 for xyz, accuracy in
          zip(data[['x', 'y', 'z']].values, data['accuracy'])]

data['s__gps_proximity_counts'] = counts

In [9]:
data['s__gps_spatial_extreme_outlier'].value_counts()

False    72473
True        61
Name: s__gps_spatial_extreme_outlier, dtype: int64

In [10]:
# DO A Further cleaning of extreme outliers by DBSCAN
coords_columns = ['x', 'y']
model = DBSCAN(eps=5, min_samples=5)  # tune these parameters for your data
model.fit(data[mask][coords_columns])
data.loc[mask, 'outlier'] = model.fit_predict(data[mask][coords_columns])
data['s__gps_spatial_extreme_outlier'] = data.apply(lambda row: 1 if row['outlier']==-1 or row['s__gps_spatial_extreme_outlier']==1 else 0, 1)


KeyboardInterrupt: 

In [ ]:
data['s__gps_spatial_extreme_outlier'].value_counts()

In [ ]:
from pyod.models.cof import COF
from pyod.models.lof import LOF

# FIND OUTLIERS WHICH ARE NOT EXTREMES

model = LOF(n_neighbors=20)
columns =  ['x', 'y'] #['f__gps_latitude', 'f__gps_longitude', 'f__gps_accuracy']#['f__gps_latitude', 'f__gps_longitude'] #
#data = data[mask][columns].drop_duplicates()
model.fit(data[mask][columns])

In [ ]:
data.loc[mask,'s__gps_spatial_outlier'] = model.predict(data[mask][columns])
#data['anomaly'] = model.labels_
#data['anomaly'] = data['anomaly'].apply(lambda x: 1 if x==-1 else 0)
data['s__gps_spatial_outlier'].value_counts()#, data['anomaly'].value_counts()/data['anomaly'].count()

In [ ]:
import folium
import matplotlib.colors

def get_color(row):
    if row['s__gps_spatial_extreme_outlier'] ==1:
        return 'red'
    elif row['s__gps_spatial_outlier'] ==1:
        return 'orange'
    elif row['s__gps_proximity_counts'] ==1:
        return 'green'
    else:
        return 'blue'

df = data
m = folium.Map(location=[df['f__gps_latitude'].mean(), df['f__gps_longitude'].mean()], zoom_start=10)


#df['color'] = df['s__proximity_counts'].apply(lambda x: 'green' if x>0 else 'blue')
df['color'] = df.apply(lambda row: get_color(row), 1)

for idx, row in df.iterrows():


    marker = folium.CircleMarker(
        location=(row['f__gps_latitude'], row['f__gps_longitude']),
        radius=5,#+row['s__proximity_counts'],
        color=row['color'],
        fill=True,
        tooltip = (row['f__gps_latitude'], row['f__gps_longitude'], row['distance_to_median']),
        fill_color=row['color']
    )

    # Add a popup label to display the value of s__spacial_outlier
    folium.Popup(f"Value: ").add_to(marker)

    marker.add_to(m)
    
m.save('map_outliers.html')

In [44]:
|m.save('map_with_labels99.html')

In [38]:
data.interview__id.count(),data.interview__id.nunique()

(976, 976)